In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import date, timedelta

from lib.evaluate.read import read_df, read_pv, read_sm
from lib.evaluate.eval import filter_df, get_usage_estimation
from lib.evaluate.view import initialize_figure, print_df_details, draw_df

today = date.today()

### Read Data

In [ ]:
folder_path = './data'

pv_df = read_df('pv', read_pv, folder_path)
sm_df = read_df('sm', read_sm, folder_path)

### Presets

In [ ]:
default_preset = 1
presets = [
    (0, 'Today', today, today + timedelta(days=1), 2),
    (1, 'Yesterday', today - timedelta(days=1), today, 2),
    (2, 'Last Week', today - timedelta(days=7), today, 7),
    (3, 'Last 2 Weeks', today - timedelta(days=14), today, 7),
    (4, 'Last 30 Days', today - timedelta(days=30), today, 7),
    (5, 'Last 90 Days', today - timedelta(days=90), today, 7),
    (6, 'All Data', date(2023, 9, 4), today, 7)
]

### Estimate PV Usage

In [ ]:
(est_usage_df, est_total_df) = get_usage_estimation(pv_df, sm_df)

### Draw Diagram

In [ ]:
fig, ax = initialize_figure()

start_date = today
end_date = today
resample = 0
show_pv = True
show_sm = True
show_est_usage = True
show_est_total = False
cumulative = False

pv_title = 'PV Production'
sm_title = 'Smartmeter Usage'
est_usage_title = 'Estimated PV Usage'
est_total_title = 'Estimated Total Usage'
cumulative_title = 'Cumulative'

presets_picker = widgets.Dropdown(
    description='Presets',
    value=default_preset,
    options=map(lambda d: (d[1], d[0]), presets),
)
from_picker = widgets.DatePicker(
    description='From',
    value=start_date,
    disabled=False
)
to_picker = widgets.DatePicker(
    description='To',
    value=end_date,
    disabled=False
)
plus_day_button = widgets.Button(
    icon='plus'
)
minus_day_button = widgets.Button(
    icon='minus'
)
resample_options = [
    (0, '5m', '5T'),
    (1, '10m', '10T'),
    (2, '15m', '15T'),
    (3, '30m', '30T'),
    (4, '1h', '1H'),
    (5, '6h', '6H'),
    (6, '12h', '12H'),
    (7, '1D', '1D'),
    (8, '1W', '1W'),
    (9, '1M', '1M')
]
resample_picker = widgets.Dropdown(
    description='Resample',
    value=resample,
    options=map(lambda d: (d[1], d[0]), resample_options)
)
show_pv_picker = widgets.ToggleButton(
    value=show_pv,
    description=pv_title
)
show_sm_picker = widgets.ToggleButton(
    value=show_sm,
    description=sm_title
)
show_est_usage_picker = widgets.ToggleButton(
    value=show_est_usage,
    description=est_usage_title
)
show_est_total_picker = widgets.ToggleButton(
    value=show_est_total,
    description=est_total_title
)
cumulative_picker = widgets.ToggleButton(
    value=cumulative,
    description=cumulative_title
)
output_detail = widgets.HTML()
output_plot = widgets.Output()

def update():
    resample_value = resample_options[resample][2]
    _pv_df = filter_df(pv_df, start_date, end_date, resample_value)
    _sm_df = filter_df(sm_df, start_date, end_date, resample_value)
    _est_usage_df = filter_df(est_usage_df, start_date, end_date, resample_value)
    _est_total_df = filter_df(est_total_df, start_date, end_date, resample_value)
    print_df_details(output_detail, _pv_df, _sm_df, _est_usage_df, _est_total_df, pv_title, sm_title, est_usage_title, est_total_title)
    if cumulative:
        draw_df(ax, _pv_df.cumsum(), _sm_df.cumsum(), _est_usage_df.cumsum(), _est_total_df.cumsum(), show_pv, show_sm, show_est_usage, show_est_total, pv_title, sm_title, est_usage_title, est_total_title)
    else:
        draw_df(ax, _pv_df, _sm_df, _est_usage_df, _est_total_df, show_pv, show_sm, show_est_usage, show_est_total, pv_title, sm_title, est_usage_title, est_total_title)

def update_presets(value):
    global start_date
    global end_date
    global resample
    
    selected = presets[value]

    from_picker.value = selected[2]
    start_date = selected[2]

    to_picker.value = selected[3]
    end_date = selected[3]

    resample_picker.value = selected[4]
    resample = selected[4]
presets_picker.observe(lambda v: update_presets(v.new), 'value')

def update_from(value):
    global start_date
    start_date = value
    update()
from_picker.observe(lambda v: update_from(v.new), 'value')

def update_to(value):
    global end_date
    end_date = value
    update()
to_picker.observe(lambda v: update_to(v.new), 'value')

def plus_day(value):
    to_picker.value = to_picker.value + timedelta(days=1)
    from_picker.value = from_picker.value + timedelta(days=1)
plus_day_button.on_click(plus_day)

def minus_day(value):
    from_picker.value = from_picker.value - timedelta(days=1)
    to_picker.value = to_picker.value - timedelta(days=1)
minus_day_button.on_click(minus_day)

def update_resample(value):
    global resample
    resample = value
    update()
resample_picker.observe(lambda v: update_resample(v.new), 'value')

def update_show_pv(value):
    global show_pv
    show_pv = value
    update()
show_pv_picker.observe(lambda v: update_show_pv(v.new), 'value')

def update_show_sm(value):
    global show_sm
    show_sm = value
    update()
show_sm_picker.observe(lambda v: update_show_sm(v.new), 'value')

def update_show_est_usage(value):
    global show_est_usage
    show_est_usage = value
    update()
show_est_usage_picker.observe(lambda v: update_show_est_usage(v.new), 'value')

def update_show_est_total(value):
    global show_est_total
    show_est_total = value
    update()
show_est_total_picker.observe(lambda v: update_show_est_total(v.new), 'value')

def update_cumulative(value):
    global cumulative
    cumulative = value
    update()
cumulative_picker.observe(lambda v: update_cumulative(v.new), 'value')

update_presets(default_preset)

button_layout = widgets.Layout(
    width='33%'
)
minus_day_button.layout = button_layout
plus_day_button.layout = button_layout
days_controls = widgets.HBox([widgets.Label('Day Modifiers'), minus_day_button, plus_day_button])
days_controls.layout = widgets.Layout(
    display='flex',
    justify_content='space-between',
    align_items='center'
)
picker_layout = widgets.Layout(
    width='95%'
)
show_pv_picker.layout = picker_layout
show_sm_picker.layout = picker_layout
show_est_usage_picker.layout = picker_layout
show_est_total_picker.layout = picker_layout
cumulative_picker.layout = picker_layout
show_controls = widgets.VBox([show_pv_picker, show_sm_picker, show_est_usage_picker, show_est_total_picker, widgets.Label(), cumulative_picker])
show_controls.layout = widgets.Layout(
    width='14em'
)
controls = widgets.VBox([presets_picker, widgets.Label(), days_controls, from_picker, to_picker, widgets.Label(), resample_picker])
header = widgets.HBox([controls, show_controls, output_detail])
header.layout = widgets.Layout(
    width='100em',
    display='flex',
    justify_content='space-around',
    align_items='center'
)
widgets.VBox([output_plot, header])
